Cellpose Training arena

This notebook includes code to train models and collect data to crossvalidate across learning rate and epoch. Currently it outputs our selected Cellpose final model




In [17]:
%pip install "opencv-python-headless<4.3"
%pip install cellpose


186277.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
186284.47s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.
186291.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Note: you may need to restart the kernel to use updated packages.


If you run in colab you can check for the GPU being activated. 
You can also run without GPU but it is slighly slower.


In [18]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob
import pandas as pd

# model name and path
from cellpose import models
from stardist.matching import matching, matching_dataset

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')


186423.23s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
zsh:1: command not found: nvcc
186428.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
zsh:1: command not found: nvidia-smi
2023-12-19 21:01:30,227 [INFO] TORCH CUDA version not installed/working.
>>> GPU activated? NO


***NOTE ON TRAINING LOSS DATA***


Initialize the logger to see training loss accross epochs. We parsed the logs manually as we had issue obtaining cellpose's run.log with the full cross validation metrics. The runtime logs are also not visible on Google Colab and there is not tensorboard support. You can find logs on our repo as mip_full_log.txt and focus_full_log.txt 

In [22]:
# these are the training images from human in the loop workflow/ the folder should contain the image and the corresponding segmentation file
# for example square_0.tif and square_0_seg.npy
train_dir = "cellposetraining/traindataHIL_mip"
test_dir = None
# images to validate, our ground truth cut in 400x400 squares
dir = "cellposetraining/mip_validate"
files = io.get_image_files(dir, '_mask')
test_data = [io.imread(f) for f in files]
# our ground truth masks
maskdir = "cellposetraining/mip_validate_mask"
gtfiles = io.get_image_files(maskdir, '')
test_labels = [io.imread(m) for m in gtfiles]

#specify the base model we will train from
initial_base_model = "cyto"
#Default model param
weight_decay = 0.0001
#Channels for our grayscale images
channels =[0, 0]

#You can add more cross validation parameters for number of epochs and learning rate.
# Make sure you update the name given to each model if you compute many models. Right now it will produce the best cellpose model we selected.

n_epochs =  [90]
learning_rates = [0.01]
#place holder, see inside the loop for updates on model_name
model_name = "CP_tissuenet"

# start logger (to see training across epochs)/ need to manually parse the output -.-
# as the logging happens within the training and there are no fn to retrieve this data
logger = io.logger_setup() #CANT SEE RUNTIME TERMINAL ON COLAB, RUN LOCALLY TO GET TRAINING LOSSES !!


for l_r in learning_rates:
  for n_e in n_epochs:
    # replace model_name with line below to costumize the model name if you are training various models with different learning rate and number of epochs
    # model_name = f"CP_mip_lr{l_r}_ep{n_e}"
    model_name = f"CP_mip_final"
    # get training files
    output = io.load_train_test_data(train_dir, test_dir, mask_filter='_seg.npy')
    train_data, train_labels, _, _, _, _ = output

    #reload test data on every iteration becasue they get modified during training
    test_data = [io.imread(f) for f in files]
    test_labels = [io.imread(m) for m in gtfiles]

    #here we check that no model with the same name already exist, if so delete
    model_path = train_dir + 'models/'
    if os.path.exists(model_path+'/'+model_name):
      print("!! WARNING: "+model_name+" already exists and will be deleted in the following call !!")

    # now let's train!
    model = models.CellposeModel(gpu=use_GPU, model_type=initial_base_model)

    print(f"#### TRAINING: {model_name} LR: {l_r} epochs: {n_e} START###")

    #TRAIN!
    new_model_path = model.train(train_data, train_labels,
                              test_data=test_data,
                              test_labels=test_labels,
                              channels=channels,
                              save_path=train_dir,
                              n_epochs=n_e,
                              learning_rate=l_r,
                              weight_decay=weight_decay,
                              nimg_per_epoch=8,
                              model_name=model_name)


    print(f"#### TRAINING: {model_name} LR: {l_r}  empochs: {n_e} END###")
    



['cellposetraining/mip_validate_mask/mip_1_0_0_mask.tif', 'cellposetraining/mip_validate_mask/mip_1_128_377_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_0_0_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_0_400_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_400_0_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_400_400_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_800_0_mask.tif', 'cellposetraining/mip_validate_mask/mip_2_800_400_mask.tif']
2023-12-20 16:37:38,775 [INFO] WRITING LOG OUTPUT TO /Users/Daniela/.cellpose/run.log
2023-12-20 16:37:38,776 [INFO] 
cellpose version: 	2.2.3 
platform:       	darwin 
python version: 	3.11.5 
torch version:  	2.0.1
2023-12-20 16:37:38,779 [INFO] not all flows are present, running flow generation for all images
2023-12-20 16:37:38,800 [INFO] 5 / 5 images in cellposetraining/traindataHIL_mip folder have labels
2023-12-20 16:37:38,815 [INFO] >> cyto << model set to be used
2023-12-20 16:37:38,816 [INFO] >>>> using CPU
20

100%|██████████| 5/5 [00:00<00:00, 18.91it/s]

2023-12-20 16:37:39,333 [INFO] computing flows for labels



100%|██████████| 8/8 [00:00<00:00, 13.13it/s]

2023-12-20 16:37:39,973 [INFO] >>>> median diameter set to = 30
2023-12-20 16:37:39,973 [INFO] >>>> mean of training label mask diameters (saved to model) 23.659
2023-12-20 16:37:39,974 [INFO] >>>> training network with 2 channel input <<<<
2023-12-20 16:37:39,974 [INFO] >>>> LR: 0.01000, batch_size: 8, weight_decay: 0.00010
2023-12-20 16:37:39,975 [INFO] >>>> ntrain = 5, ntest = 8
2023-12-20 16:37:39,975 [INFO] >>>> nimg_per_epoch = 8


2023-12-20 16:37:45,258 [INFO] Epoch 0, Time  5.3s, Loss 0.8118, Loss Test 0.8520, LR 0.0000
2023-12-20 16:37:49,108 [INFO] saving network parameters to cellposetraining/traindataHIL_mip/models/CP_mip_final
2023-12-20 16:38:04,785 [INFO] Epoch 5, Time 24.8s, Loss 0.6704, Loss Test 0.8178, LR 0.0056
2023-12-20 16:38:23,449 [INFO] Epoch 10, Time 43.5s, Loss 0.4909, Loss Test 0.8187, LR 0.0100
2023-12-20 16:39:01,452 [INFO] Epoch 20, Time 81.5s, Loss 0.4464, Loss Test 0.8343, LR 0.0100
2023-12-20 16:39:37,183 [INFO] Epoch 30, Time 117.2s, Loss 0.4629, Loss Test 0.8423, LR 0.0100
2023-12-20 16:40:12,779 [INFO] Epoch 40, Time 152.8s, Loss 0.3930, Loss Test 0.8436, LR 0.0100
2023-12-20 16:40:48,598 [INFO] Epoch 50, Time 188.6s, Loss 0.3827, Loss Test 0.8280, LR 0.0100
2023-12-20 16:41:25,366 [INFO] Epoch 60, Time 225.4s, Loss 0.4035, Loss Test 0.8180, LR 0.0100
2023-12-20 16:42:01,913 [INFO] Epoch 70, Time 261.9s, Loss 0.3823, Loss Test 0.8145, LR 0.0100
2023-12-20 16:42:36,291 [INFO] Epoch 